Process and format the electricity and gas data for the analysis.

In [ ]:
### IMPORTS

import pandas as pd

In [2]:
### INPUTS

input_dir = r"D:\Documentos\Artículos\Análisis energía España\data_sources\00_esios_ree_raw"
output_dir = r"D:\Documentos\Artículos\Análisis energía España\data_sources\01_esios_ree_processed"

In [8]:
### LOAD DATA

gas_price = pd.read_csv(f'{input_dir}\gas_price.csv')
gas_price_RD = pd.read_csv(f'{input_dir}\gas_price_RD.csv')
elec_spot_price = pd.read_csv(f'{input_dir}\price_data.csv')
power_generation = pd.read_csv(f'{input_dir}\generation_data.csv')
installed_capacity = pd.read_csv(f'{input_dir}\capacity_data.csv')

In [9]:
### PROCESS GAS PRICE

# Rename columns
gas_price = gas_price.rename(columns={'fecha': 'date','precio': 'price', 'Producto': 'indicator'})
gas_price_RD = gas_price_RD.rename(columns={'Date': 'date', 'Price(EUR/MWh)': 'price'})

# Convert datetime column to datetime object
gas_price.date = pd.to_datetime(gas_price.date)
gas_price_RD.date = pd.to_datetime(gas_price_RD.date)

# Add product column to RD gas price
gas_price_RD['indicator'] = 'PGN'

# Join dataframes
concat_gas = pd.concat([gas_price, gas_price_RD])

# Save to CSV
concat_gas.to_csv('{}\gas_price.csv'.format(output_dir),index=False)

In [ ]:
### PROCESS ELECTRICITY SPOT PRICE

# Read in data
elec_spot_price = pd.read_csv(f'{input_dir}\price_data.csv')

# Rename columns
elec_spot_price = elec_spot_price.rename(columns={'geo_name': 'country', 'value': 'price'})

# Drop unnecessary columns
elec_spot_price = elec_spot_price.drop(['datetime_utc','tz_time','geo_id','name','short_name'], axis=1)

# Set datetime as index
elec_spot_price.index = elec_spot_price.datetime

# Rename datetime column and drop original datetime column
elec_spot_price = elec_spot_price.rename(columns={'datetime':'datetime_col'}).drop('datetime_col', axis=1)

# Save selected columns to CSV
elec_spot_price[['country', 'price']].to_csv('{}\elec_spot_price.csv'.format(output_dir), index=True)

In [32]:
### PROCESS ELECTRICITY GENERATION

# Read in data
power_generation = pd.read_csv(f'{input_dir}\generation_data.csv')

# Rename columns
power_generation = power_generation.rename(columns={'value': 'generation','short_name':'source'})

# Drop unnecessary columns
power_generation = power_generation.drop(['datetime_utc','tz_time','geo_id','name','geo_name'], axis=1)

# Set datetime as index
power_generation.index = power_generation.datetime

# Rename datetime column and drop original datetime column
power_generation = power_generation.rename(columns={'datetime':'datetime_col'}).drop('datetime_col', axis=1)

# Save selected columns to CSV
power_generation[['source', 'generation']].to_csv('{}\power_generation.csv'.format(output_dir), index=True)

In [72]:
### PROCESS INSTALLED CAPACITY

# Read in data
installed_capacity = pd.read_csv(f'{input_dir}\capacity_data.csv')

# Rename columns
installed_capacity = installed_capacity.rename(columns={'short_name': 'source', 'value': 'capacity'})

# Drop unnecessary columns
installed_capacity = installed_capacity.drop(['datetime_utc','tz_time','geo_id','geo_name','name'], axis=1)

# Set datetime as index
installed_capacity.index = installed_capacity.datetime

# Rename datetime column and drop original datetime column
installed_capacity = installed_capacity.rename(columns={'datetime':'datetime_col'}).drop('datetime_col', axis=1)

# Aggregate other sources
other_sources = set(['Fuel+gas','Hidroeólica','Residuos renovables','Residuos no renovables',
                    'Turbina de vapor','Motores diesel','Otras renovables','Turbina de gas'])
installed_capacity['agg_source'] = installed_capacity['source'].apply(lambda x: 'Otras' if x in other_sources else x)

# Remove total source, drop disaggregated source and rename aggregated source to source
installed_capacity = installed_capacity[installed_capacity.source!='Total']
installed_capacity = installed_capacity.drop('source',axis=1).rename(columns={'agg_source':'source'})

# Group by new source and reset index
installed_capacity = installed_capacity.groupby(by=['datetime','source']).sum().reset_index().set_index('datetime').sort_values(by='datetime')

# Save selected columns to CSV
installed_capacity[['source', 'capacity']].to_csv('{}\installed_capacity.csv'.format(output_dir), index=True)